# Abgabe 2
Hinweise:
- Bearbeiten Sie alle Aufgaben bis zum **28.04.2025, 22 Uhr**.
- Bei Aufgaben 2 soll ein (kleines) Gurobi-Programm geschrieben werden. Reichen sie dieses bitte als Jupyter Notebook ein. Die restlichen Abgaben können sie entweder als Markdown (Jupyter), pdf oder auch als Foto einer handgeschriebenen Lösung abgeben.
- Sie können die Lösung in Gruppen bearbeiten, aber jede/r sollte eine eigene Lösung abgeben.
- Als Teil der Klausurzulassung müssen sie 50\% der Aufgaben korrekt lösen. Versuchen sie aber, bei *allen* Aufgaben zumindest Lösungsansätze zu entwickeln.


# Aufgabe 1: Zeitabhängige Modellierung
Ein Trinkwasserversorger betreibt ein Wasserreservoir, aus dem Trinkwasser zu den Verbrauchern abfließt. Die Kapazität des Reservoirs beträgt $1,000,000$ Kubikmeter. Mittels elektrischer Pumpen wird dieses wieder befüllt, allerdings können in einer Stunde maximal $70,000$ Kubikmeter gepumpt werden. Der Trinkwasserversorger möchte nun von schwankenden Energiepreisen profitieren und sein Wasserreservoir möglichst dann auffüllen, wenn der Strom günstig ist. Allerdings soll auch zu jeder Zeit der Trinkwasserbedarf der Verbraucher gedeckt sein.

Pro Kubikmeter Wasser, der in das Reservoir gepumpt wird, fällt ein Preis von $p_t$ € an. Der Planungshorizont beträgt 24 Stunden und der Strompreis wird für diese als bekannt vorausgesetzt. Die Werte seien $$p_1,p_2,\dots,p_{24}.$$

Auch für den Trinkwasserbedarf der nächsten 24 Stunden in Kubikmetern ist eine Bedarfsvorhersage 
$$d_1,d_2,\dots,d_{24}$$ 
gegeben. Es sollen nun folgende Entscheidungen getroffen werden:
\begin{align*}
	h_t^+&=\textup{Menge an Wasser (in m³), die in Stunde }t\textup{ zum aktuellen Strompreis gepumpt werden soll},\quad t=1,\dots,24.
\end{align*}
Den Wasserstand (in m³) in jeder Stunde bezeichnen wir mit $$h_t,\quad t=1,\dots,24.$$ Sie können annehmen, dass das Reservoir zu Beginn der betrachteten Zeitperiode $50,000$ Kubikmeter Wasser enthält.
Die Entscheidungen sollen so getroffen werden, dass die Gesamtstromkosten möglichst gering sind und gleichzeitig der Bedarf an Trinkwasser gedeckt wird. Des weiteren soll der Wasserstand im Reservoir nie unter 20,000 Kubikmeter sinken.

1. Geben Sie die mathematische Formulierung der Zielfunktion an.
2. Formulieren Sie sinnvolle obere und untere Grenzen für *alle* Variablen des Optimierungsproblems.
3. Formulieren Sie die (Gleichungs-)Nebenbedingungen des Problems.
4. Wenn sie das Optimierungsproblem in Standardschreibweise aufschreiben würden, wie viele Zeilen und Spalten hätte dann die Matrix $\mathbf{A}$ der Gleichungsnebenbedingungen?
5. Warum wäre die optimale Lösung des Modells in der Realität zu optimistisch (d.h. in der Realität wären die Kosten wahrscheinlich höher)?


# Aufgabe 2: Implementierung des zeitabhängigen Problems
Für die nächsten 24 Stunden sind folgende Werte gegeben:

In [35]:
import pandas as pd

df = pd.DataFrame({"Preis [€/m³]": [0.34,0.58,0.60,0.14,0.21,0.24,0.124,0.50,0.44,0.56,0.35,0.77,0.24,0.44,
                                    0.50,0.12,0.44,0.45,0.42,0.62,0.51,0.49,0.51,0.76],
                   "Bedarf [m³]": [70000,60000,50000,40000,80000,60000,60000,80000,52000,54000,66000, 51000,23000,
                              20000,86000,58000,54000,60000,64000,62000,48000,71000,24000,12000]},
                       index=list(range(1,25)))

Lösen Sie das Modell mit Gurobi, geben Sie die optimalen Kosten aus und visualisieren Sie die Lösung. 


# Aufgabe 3: Optimierung auf rollierenden Zeitfenstern


Beschreiben Sie, wie man die in Abschnitt 4.3 im Skript beschriebene *Sequenzielle Optimierung auf rollierendem Zeitfenster* auf das Problem der vorherigen Aufgaben anwenden würde. Nehmen Sie dabei an, dass die einzige Unsicherheit der Trinkwasserbedarf ist, für den sie in jeder Zeitperiode eine neue Vorhersage bekommen.  

___

# Aufgabe 1

Wir verwenden **Zeitdiskretisierung**: Der Tag wird in 24 einstündige Intervalle $t=1, \dots, 24$ unterteilt.

## Definitionen

* **Indexmenge (Zeithorizont):** $H = \{1, 2, \dots, 24\}$
* **Problemdaten:**
    * Strompreis pro m³ gepumptem Wasser in Stunde $t$: $p_t$ (€/m³) für $t \in H$.
    * Wasserbedarf in Stunde $t$: $d_t$ (m³) für $t \in H$.
    * Maximale Reservoirkapazität: $h_{max} = 1,000,000$ m³.
    * Minimaler Reservoirfüllstand: $h_{min} = 20,000$ m³.
    * Maximale Pumpleistung pro Stunde: $h_{pump\_max} = 70,000$ m³.
    * Initialer Füllstand (Ende Stunde $t=0$): $h_0 = 50,000$ m³.
* **Entscheidungsvariable:**
    * Gepumpte Wassermenge in Stunde $t$: $h_t^+$ (m³) für $t \in H$.
    * Wasserstand im Reservoir am *Ende* der Stunde $t$: $h_t$ (m³) für $t \in H$.

---

### Aufgabe 1.1: Zielfunktion


**Antwort:**
Ziel ist die Minimierung der Gesamtstromkosten über 24 Stunden. Die Kosten in Stunde $t$ sind $p_t \cdot h_t^+$. Die Summe über alle Stunden soll minimiert werden.

$$
\min_{h_t^+, h_t} \sum_{t=1}^{24} p_t h_t^+
$$

---

### Aufgabe 1.2: Variable Grenzen

**Antwort:**
Wir betrachten die Entscheidungsvariable $h_t^+$ und die Zustandsvariable $h_t$.

* **Grenzen für die gepumpte Menge $h_t^+$:**
    * Nicht-Negativität: $h_t^+ \ge 0$.
    * Maximale Pumpleistung: $h_t^+ \le 70,000$.
    * Zusammengefasst:
        $$
        0 \le h_t^+ \le 70,000 \quad \forall t = 1, \dots, 24
        $$

* **Grenzen für den Wasserstand $h_t$:**
    * Minimaler Füllstand: $h_t \ge 20,000$.
    * Maximale Kapazität: $h_t \le 1,000,000$.
    * Zusammengefasst:
        $$
        20,000 \le h_t \le 1,000,000 \quad \forall t = 1, \dots, 24
        $$

---

### Aufgabe 1.3: Gleichungsnebenbedingungen

**Antwort:**
Die zentrale Gleichungsnebenbedingung ist die **Wasserbilanzgleichung**. Sie beschreibt, wie sich der Wasserstand $h_t$ am Ende der Stunde $t$ aus dem Stand der Vorstunde $h_{t-1}$, der zugepumpten Menge $h_t^+$ und dem entnommenen Bedarf $d_t$ ergibt.

Die Bilanzgleichung für jede Stunde $t = 1, \dots, 24$:
$$
h_t = h_{t-1} + h_t^+ - d_t \quad \forall t = 1, \dots, 24
$$
Für $t=1$ wird der feste Wert $h_0 = 50,000$ verwendet. Diese Gleichung stellt sicher, dass die Wasserflüsse korrekt bilanziert werden und implizit (zusammen mit den Füllstandsgrenzen) die Bedarfsdeckung ermöglicht wird.

---

### Aufgabe 1.4: Dimension der Matrix A

**Antwort:**
Die Matrix $\mathbf{A}$ repräsentiert die Koeffizienten der Variablen in den *Gleichungs*nebenbedingungen ($Ax=b$).

1.  **Variablen ($x$):**
    * $h_t^+$ für $t=1, \dots, 24 \implies 24$ Variablen
    * $h_t$ für $t=1, \dots, 24 \implies 24$ Variablen
    * Gesamtzahl der Variablen: $N = 24 + 24 = 48$. Dies ist die Anzahl der Spalten von $\mathbf{A}$.

2.  **Gleichungsnebenbedingungen ($Ax=b$):**
    * Die Wasserbilanzgleichung $h_t = h_{t-1} + h_t^+ - d_t$ (umgeformt $h_t - h_{t-1} - h_t^+ = -d_t$) ist die einzige Art von Gleichungsnebenbedingung.
    * Es gibt eine solche Gleichung für jede Stunde $t = 1, \dots, 24$.
    * Gesamtzahl der Gleichungen: $M = 24$. Dies ist die Anzahl der Zeilen von $\mathbf{A}$.

Die Matrix $\mathbf{A}$ hat also **24 Zeilen** und **48 Spalten**.

---

### Aufgabe 1.5: Optimismus der Modelllösung

**Antwort:**
Die Diskrepanz zwischen der optimalen Lösung des *Modells* und dem Ergebnis in der *Realität* entsteht hauptsächlich durch die Vereinfachungen und Annahmen des Modells, wie in Abschnitt 4.3 des Skripts diskutiert. Hauptgründe:

1.  **Unsichere Prognosen:**
    * Das Modell nimmt an, dass die zukünftigen Strompreise ($p_t$) und der Wasserbedarf ($d_t$) für alle 24 Stunden *exakt bekannt* sind (deterministische Sicht).
    * In Wirklichkeit sind dies Prognosen, die fehlerbehaftet sind. Abweichungen der realen Werte von den Prognosen führen dazu, dass der berechnete "optimale" Plan eventuell nicht mehr optimal oder sogar teurer ist, wenn er mit den realen Daten konfrontiert wird. Die Modelllösung basiert auf einer perfekten Zukunftskenntnis, die nicht existiert.

2.  **Modellvereinfachungen:**
    * **Effizienz:** Das Modell berücksichtigt keine Energieverluste beim Pumpvorgang (z.B. durch Reibung, Wärme). Real benötigt man eventuell mehr Energie (höhere Kosten) als im Modell berechnet, um $h_t^+$ Kubikmeter zu pumpen.
    * **Konstanz:** Annahme konstanter Werte ($p_t$, $d_t$) innerhalb einer Stunde, obwohl sie sich in Realität kontinuierlich ändern könnten.
    * **Andere Faktoren:** Mögliche Vernachlässigung anderer Betriebsaspekte wie Wartung, variable Pumpeneffizienz etc.

Da das Modell das Optimum unter idealisierten Bedingungen (perfekte Voraussicht, keine Verluste) berechnet, stellt der resultierende Zielfunktionswert (minimale Kosten) eine theoretische untere Schranke dar. Die in der Realität anfallenden Kosten werden aufgrund von Unsicherheiten und Modellungenauigkeiten typischerweise höher sein. Ansätze wie die *Sequenzielle Optimierung auf rollierendem Zeitfenster* (Model Predictive Control - MPC) versuchen, diesem Problem durch wiederholte Optimierung mit aktualisierten Daten entgegenzuwirken.

___
## Aufgabe 2

In [4]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB 

df = pd.DataFrame({"Preis [€/m³]": [0.34,0.58,0.60,0.14,0.21,0.24,0.124,0.50,0.44,0.56,0.35,0.77,0.24,0.44,
                                    0.50,0.12,0.44,0.45,0.42,0.62,0.51,0.49,0.51,0.76],
                   "Bedarf [m³]": [70000,60000,50000,40000,80000,60000,60000,80000,52000,54000,66000, 51000,23000,
                              20000,86000,58000,54000,60000,64000,62000,48000,71000,24000,12000]},
                       index=list(range(1,25)))

prices = df["Preis [€/m³]"].to_dict()
demands = df["Bedarf [m³]"].to_dict()

h_max = 1000000
h_min = 20000
h_pump_max = 70000
h_0 = 50000
T = list(range(1, 25)) # Zeitindizes 1 bis 24


model = gp.Model("Wasserreservoir_Optimierung")

# Optional: Gurobi-Ausgaben unterdrücken (Standard ist an)
model.setParam('OutputFlag', 0)

# -- 3. Entscheidungsvariablen definieren --
# Gepumpte Menge h_t^+
# addVars(Indizes, untere Grenze, obere Grenze, Variablentyp, Name)
h_plus = model.addVars(T, lb=0, ub=h_pump_max, vtype=GRB.CONTINUOUS, name="Pumpmenge")

# Wasserstand h_t
# addVars(Indizes, untere Grenze, obere Grenze, Variablentyp, Name)
h = model.addVars(T, lb=h_min, ub=h_max, vtype=GRB.CONTINUOUS, name="Fuellstand")

# -- 4. Zielfunktion definieren --
# quicksum ist effizienter als eine Python-Schleife
objective = gp.quicksum(prices[t] * h_plus[t] for t in T)
model.setObjective(objective, GRB.MINIMIZE)

# -- 5. Nebenbedingungen hinzufügen --
# Wasserbilanzgleichung: h_t = h_{t-1} + h_t^+ - d_t
# h[0] gibt es nicht als Variable, wir müssen h_0 direkt einsetzen
model.addConstr(h[1] == h_0 + h_plus[1] - demands[1], name=f"Bilanz_Stunde_1")

# Mit addConstrs und Generator Expression:
model.addConstrs((h[t] == h[t-1] + h_plus[t] - demands[t] for t in range(2, 25)), name="Bilanz_Stunde")


# -- 6. Modell optimieren --
model.optimize()

# -- 7. Ergebnisse ausgeben --
print("-" * 30)
if model.Status == GRB.OPTIMAL:
    print(f"Optimale Gesamtkosten: {model.ObjVal:.2f} €")
    print("-" * 30)
    print("Optimale Pumpmengen [m³/h]:")

    results = []
    for t in T:
        # Zugriff auf den Wert einer Variablen mit .X
        pump_amount = h_plus[t].X
        reservoir_level = h[t].X
        print(f"Stunde {t:2d}: Pumpen={pump_amount:8.1f}, Füllstand Ende={reservoir_level:9.1f}")
        results.append({'Stunde': t,
                        'Preis': prices[t],
                        'Bedarf': demands[t],
                        'Pumpmenge': pump_amount,
                        'Fuellstand': reservoir_level})
else:
     print(f"Optimierung beendet mit Status Code: {model.Status}")


------------------------------
Optimale Gesamtkosten: 468580.00 €
------------------------------
Optimale Pumpmengen [m³/h]:
Stunde  1: Pumpen= 70000.0, Füllstand Ende=  50000.0
Stunde  2: Pumpen= 70000.0, Füllstand Ende=  60000.0
Stunde  3: Pumpen= 10000.0, Füllstand Ende=  20000.0
Stunde  4: Pumpen= 70000.0, Füllstand Ende=  50000.0
Stunde  5: Pumpen= 70000.0, Füllstand Ende=  40000.0
Stunde  6: Pumpen= 70000.0, Füllstand Ende=  50000.0
Stunde  7: Pumpen= 70000.0, Füllstand Ende=  60000.0
Stunde  8: Pumpen= 70000.0, Füllstand Ende=  50000.0
Stunde  9: Pumpen= 70000.0, Füllstand Ende=  68000.0
Stunde 10: Pumpen= 53000.0, Füllstand Ende=  67000.0
Stunde 11: Pumpen= 70000.0, Füllstand Ende=  71000.0
Stunde 12: Pumpen=     0.0, Füllstand Ende=  20000.0
Stunde 13: Pumpen= 70000.0, Füllstand Ende=  67000.0
Stunde 14: Pumpen= 70000.0, Füllstand Ende= 117000.0
Stunde 15: Pumpen= 70000.0, Füllstand Ende= 101000.0
Stunde 16: Pumpen= 70000.0, Füllstand Ende= 113000.0
Stunde 17: Pumpen= 70000.0,

___

# Aufgabe 3: Optimierung auf rollierenden Zeitfenstern

Die Anwendung der *Sequenziellen Optimierung auf rollierendem Zeitfenster* (auch bekannt als Model Predictive Control, MPC) auf das Wasserreservoir-Problem mit unsicherem Bedarf $d_t$ bedeutet, dass wir den Optimierungsplan nicht nur einmal für 24 Stunden erstellen, sondern diesen Prozess **jede Stunde neu starten**. Dabei verwenden wir die jeweils aktuellsten verfügbaren Informationen (insbesondere die neueste Bedarfsprognose und den aktuellen Füllstand) und implementieren **nur die Entscheidung für die unmittelbar nächste Stunde**.

## Notation

* $\tau$: Die aktuelle "Echtzeit"-Stunde ($\tau = 0, 1, 2, \dots, 23$). Der Zeitpunkt $\tau$ markiert den Beginn der Stunde $\tau+1$.
* $h_{actual, \tau}$: Der **tatsächliche** Füllstand des Reservoirs am Ende der Stunde $\tau$ (oder Beginn der Stunde $\tau+1$). Dies ist der Startzustand für die Optimierung bei Schritt $\tau$. ($h_{actual, 0} = 50,000$).
* $d_{\tau, \tau+t}$: Die **Bedarfsprognose** für die zukünftige Stunde $\tau+t$ ($t=1, \dots, 24$), die zum Zeitpunkt $\tau$ verfügbar ist.
* $p_{\tau+t}$: Der bekannte (oder ebenfalls prognostizierte) Strompreis für die zukünftige Stunde $\tau+t$.
* $h_{\tau, \tau+t}^+$: Die **geplante** Pumpmenge für Stunde $\tau+t$, berechnet zum Zeitpunkt $\tau$ für den Horizont $t=1..24$.
* $h_{\tau, \tau+t}$: Der **geplante** Füllstand am Ende der Stunde $\tau+t$, berechnet zum Zeitpunkt $\tau$ für den Horizont $t=1..24$.
* $h_{impl, \tau+1}^+$: Die **tatsächlich implementierte** Pumpmenge für Stunde $\tau+1$.
* $d_{actual, \tau+1}$: Der **tatsächlich eingetretene** Bedarf in Stunde $\tau+1$.

## Ablauf der Sequenziellen Optimierung (Iterativ für $\tau = 0, \dots, 23$)

Für jede Stunde $\tau$ von 0 bis 23 werden die folgenden Schritte durchgeführt:

1.  **Aktuellen Zustand erfassen:**
    Messe oder bestimme den aktuellen, tatsächlichen Füllstand des Reservoirs: $h_{actual, \tau}$.
    *(Für $\tau=0$: $h_{actual, 0} = 50,000$)*.

2.  **Aktuelle Prognose holen:**
    Erhalte die neueste Bedarfsprognose für die nächsten 24 Stunden: $d_{\tau, \tau+1}, d_{\tau, \tau+2}, \dots, d_{\tau, \tau+24}$.

3.  **Optimierungsproblem aufstellen (Horizont: nächste 24h):**
    * Erstelle ein Gurobi-Modell (oder ein anderes LP-Modell).
    * **Variablen:** Definiere Optimierungsvariablen für den Planungshorizont $t = 1, \dots, 24$:
        * Pumpmenge: $h_{\tau, \tau+t}^+$
        * Füllstand: $h_{\tau, \tau+t}$
    * **Zielfunktion (Minimiere erwartete Kosten):**
        $$
        \min \sum_{t=1}^{24} p_{\tau+t} \cdot h_{\tau, \tau+t}^+
        $$
    * **Nebenbedingungen:**
        * **Variablengrenzen:**
            $$
            0 \le h_{\tau, \tau+t}^+ \le h_{pump\_max} \quad \forall t = 1, \dots, 24
            $$
            $$
            h_{min} \le h_{\tau, \tau+t} \le h_{max} \quad \forall t = 1, \dots, 24
            $$
        * **Wasserbilanz:**
            * Für $t=1$ (Start mit *aktuellem* Füllstand und *neuester* Prognose):
                $$
                h_{\tau, \tau+1} = h_{actual, \tau} + h_{\tau, \tau+1}^+ - d_{\tau, \tau+1}
                $$
            * Für $t=2, \dots, 24$:
                $$
                h_{\tau, \tau+t} = h_{\tau, \tau+t-1} + h_{\tau, \tau+t}^+ - d_{\tau, \tau+t}
                $$

4.  **Optimierungsproblem lösen:**
    * Löse das in Schritt 3 aufgestellte LP-Problem (z.B. mit `model.optimize()` in Gurobi).
    * Überprüfe, ob eine optimale Lösung gefunden wurde.
    * Extrahiere die optimale **geplante** Pumpmenge für die **erste Stunde** des Plans: $h_{\tau, \tau+1}^{*+} = \text{Wert von } h_{\tau, \tau+1}^+ \text{ in der Lösung}$.

5.  **Entscheidung implementieren:**
    * Führe **nur** die Aktion für die nächste Stunde $\tau+1$ aus: Setze die tatsächliche Pumpmenge $h_{impl, \tau+1}^+$ auf den Wert $h_{\tau, \tau+1}^{*+}$. Der Rest des 24-Stunden-Plans wird verworfen.

6.  **Systemzustand aktualisieren:**
    * Die Zeit schreitet eine Stunde voran ($\tau \rightarrow \tau+1$).
    * Der **tatsächliche** Bedarf $d_{actual, \tau+1}$ für die gerade vergangene Stunde wird bekannt (dieser kann von der Prognose $d_{\tau, \tau+1}$ abweichen!).
    * Berechne den **neuen tatsächlichen** Füllstand am Ende der Stunde $\tau+1$:
        $$
        h_{actual, \tau+1} = h_{actual, \tau} + h_{impl, \tau+1}^+ - d_{actual, \tau+1}
        $$
    * Dieser neue Füllstand $h_{actual, \tau+1}$ ist der Startpunkt für die nächste Iteration.

7.  **Nächste Iteration:**
    Wiederhole die Schritte 1-6 für den nächsten Zeitpunkt $\tau+1$, solange $\tau < 23$.

## Ergebnis der Strategie

Das Endergebnis dieser Vorgehensweise ist nicht ein einzelner 24-Stunden-Plan, sondern die Sequenz der **tatsächlich ausgeführten** stündlichen Pumpmengen:
$$
h_{impl, 1}^+, h_{impl, 2}^+, \dots, h_{impl, 24}^+
$$

